# BlokusAI - Google Colab トレーニング

このノートブックでは、BlokusAI を Google Colab の GPU/TPU で高速にトレーニングできます。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/BlokusAI/blob/main/notebooks/BlokusAI_Colab_Training.ipynb)

## 🚀 クイックスタート

1. **Runtime → Change runtime type → GPU** を選択
2. 以下のセルを順番に実行
3. トレーニング開始！

---

## 📋 ステップ 1: セットアップ

リポジトリのクローンと依存関係のインストール

In [ ]:
# リポジトリをクローン
!git clone https://github.com/YOUR_USERNAME/BlokusAI.git
%cd BlokusAI

In [ ]:
# GPU対応 PyTorch をインストール (CUDA 12.1)
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121

# その他の依存関係
!pip install -q numpy matplotlib wandb python-dotenv

print("✅ インストール完了！")

## 🔍 ステップ 2: デバイス確認

GPU/TPU が正しく検出されているか確認

In [ ]:
import torch

# GPU が利用可能か確認
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

# BlokusAI のデバイス検出を確認
from blokus_ai.device import get_device_name
print(f"\n✅ Selected device: {get_device_name()}")

## 🎮 ステップ 3: トレーニング実行

以下のいずれかを選択して実行してください。

### オプション 1: クイックテスト（2分程度）

動作確認用の短時間トレーニング

In [ ]:
from blokus_ai.train import main

# クイックテスト（2イテレーション、評価なし）
main(
    num_iterations=2,
    games_per_iteration=5,
    num_simulations=15,
    eval_interval=10  # 評価はスキップ
)

print("\n✅ テスト完了！")

### オプション 2: クイックトレーニング（10-15分程度）

6イテレーション、過去モデル対戦あり

In [ ]:
from blokus_ai.train import main

# クイックトレーニング
main(
    num_iterations=6,
    games_per_iteration=10,
    num_simulations=30,
    eval_interval=2,
    past_generations=[2]  # 2世代前と対戦
)

print("\n✅ クイックトレーニング完了！")

### オプション 3: 標準トレーニング（30-45分程度）

50イテレーション、5世代・10世代前と対戦

In [ ]:
from blokus_ai.train import main

# 標準トレーニング（GPU推奨）
main(
    num_iterations=50,
    games_per_iteration=20,
    num_simulations=50,
    eval_interval=5,
    past_generations=[5, 10]  # 5世代前・10世代前と対戦
)

print("\n✅ 標準トレーニング完了！")

### オプション 4: フルトレーニング（1-2時間程度）

100イテレーション、最大パフォーマンス設定

In [ ]:
from blokus_ai.train import main

# フルトレーニング（GPU必須、A100推奨）
main(
    num_iterations=100,
    games_per_iteration=30,
    num_simulations=100,
    eval_interval=10,
    past_generations=[10, 20, 50],  # 複数世代前と対戦
    batch_size=16,
    learning_rate=0.001,
    max_grad_norm=1.0
)

print("\n✅ フルトレーニング完了！")

## 📊 オプション: WandB ロギング

トレーニングの進捗をリアルタイムで可視化

In [ ]:
import wandb

# WandB にログイン（APIキーが必要）
wandb.login()

print("✅ WandB ログイン完了！")

In [ ]:
from blokus_ai.train import main

# WandB を有効にしてトレーニング
main(
    num_iterations=50,
    games_per_iteration=20,
    num_simulations=50,
    eval_interval=5,
    use_wandb=True  # WandBロギングを有効化
)

print("\n✅ トレーニング完了！WandB ダッシュボードで確認できます。")

## 🎯 ステップ 4: モデル評価

トレーニング済みモデルの性能を評価

In [ ]:
# ベースライン評価（Random vs Greedy）
!python -m blokus_ai.eval

In [ ]:
import torch
from blokus_ai.eval import evaluate_net
from blokus_ai.net import PolicyValueNet

# 最新モデルをロード
net = PolicyValueNet()
net.load_state_dict(torch.load("models/blokus_model.pth", map_location="cpu"))

# カスタム評価
evaluate_net(
    net,
    num_games=30,
    num_simulations=50
)

## 🎨 ステップ 5: 可視化

AIの思考プロセスを可視化

In [ ]:
# 可視化デモを実行
!python scripts/demo_viz.py

In [ ]:
from IPython.display import Image, display

# 生成された可視化画像を表示
print("MCTS Top-5 手の分析:")
display(Image("game_analysis/demo_mcts_topk.png"))

print("\n確率分布ヒートマップ:")
display(Image("game_analysis/demo_heatmap.png"))

## 💾 ステップ 6: モデルのダウンロード

トレーニング済みモデルをローカルに保存

In [ ]:
from google.colab import files

# 最新モデルをダウンロード
files.download('models/blokus_model.pth')

print("✅ モデルダウンロード完了！")

In [ ]:
# 特定のチェックポイントをダウンロード（イテレーション番号を指定）
iteration = 50  # ダウンロードしたいイテレーション番号

checkpoint_path = f'models/checkpoints/checkpoint_iter_{iteration:04d}.pth'
files.download(checkpoint_path)

print(f"✅ イテレーション {iteration} のチェックポイントをダウンロード完了！")

### オプション: Google Drive に保存

複数のチェックポイントを一度に保存

In [ ]:
from google.colab import drive

# Google Drive をマウント
drive.mount('/content/drive')

In [ ]:
# モデルを Drive にコピー
!mkdir -p /content/drive/MyDrive/BlokusAI
!cp models/blokus_model.pth /content/drive/MyDrive/BlokusAI/
!cp -r models/checkpoints /content/drive/MyDrive/BlokusAI/
!cp -r game_analysis /content/drive/MyDrive/BlokusAI/

print("✅ 全てのファイルを Google Drive に保存完了！")

## 🔧 トラブルシューティング

### GPU が検出されない場合

1. **Runtime → Change runtime type → GPU** を選択
2. ランタイムを再起動
3. PyTorch を再インストール:
   ```bash
   !pip uninstall torch torchvision -y
   !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
   ```

### メモリ不足エラー (Out of Memory)

以下のパラメータを調整:

```python
main(
    batch_size=8,           # デフォルト 16 → 8 に減らす
    num_simulations=30,     # デフォルト 50 → 30 に減らす
    games_per_iteration=10  # デフォルト 20 → 10 に減らす
)
```

### トレーニングが遅い

**正常な速度:**
- GPU (T4): 1 iteration = 30-60秒
- CPU: 1 iteration = 3-5分

GPU を使用しているのに遅い場合:
- `get_device_name()` で GPU が検出されているか確認
- `nvidia-smi` で GPU 使用率を確認

---

## 📚 詳細情報

- [README.md](../README.md) - プロジェクト概要
- [TRAINING_GUIDE.md](../docs/TRAINING_GUIDE.md) - トレーニング詳細ガイド
- [COLAB_SETUP.md](../docs/COLAB_SETUP.md) - Colab セットアップ詳細
- [VISUALIZATION.md](../docs/VISUALIZATION.md) - 可視化ガイド

## 📞 サポート

問題があれば GitHub Issues で報告してください。

Happy training! 🎮🤖